# Notebook 1 – Exploración de Datos (EDA) con Datos de Accidentes de Tránsito

**Objetivo del notebook:**  
Aprender a realizar una exploración de datos (EDA) básica y limpieza inicial usando un dataset real sobre accidentes de tránsito y seguros en distintos estados de EE. UU.

**Dataset:**  
Cada fila representa un estado y contiene indicadores de choques fatales y costos de seguros:

- `Estado`
- `Número de conductores involucrados en colisiones fatales por cada mil millones de millas`
- `Porcentaje de conductores involucrados en colisiones fatales con exceso de velocidad`
- `Porcentaje de conductores involucrados en colisiones fatales bajo la influencia del alcohol`
- `Porcentaje de conductores involucrados en colisiones fatales que no estaban distraídos`
- `Porcentaje de conductores involucrados en colisiones fatales que no habían estado involucrados en accidentes previos`
- `Primas de seguro de automóvil ($)`
- `Pérdidas incurridas por las compañías de seguros por colisiones por conductor asegurado ($)`

En este notebook vamos a:

1. Importar el dataset y entender su estructura.
2. Revisar tipos de datos, valores faltantes y duplicados.
3. Detectar outliers numéricos de forma simple.
4. Crear algunas variables nuevas que tengan sentido analítico.
5. Dejar un **dataframe limpio y listo** para usar en modelos en el siguiente notebook.



In [47]:
# ============================================================
# 1. Importar librerías y cargar el dataset
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Opciones de visualización
pd.set_option('display.max_columns', None)
plt.style.use('default')

# Cargar el dataset (ajusta la ruta al archivo si es necesario)
# Por ejemplo: 'bad-drivers.csv' en la misma carpeta del notebook
df = pd.read_csv("bad-drivers.csv")

# Mirar las primeras filas
df.head()

,Estado,Número de conductores involucrados en colisiones fatales por cada mil millones de millas,Porcentaje de conductores involucrados en colisiones fatales con exceso de velocidad,Porcentaje de conductores involucrados en colisiones fatales bajo la influencia del alcohol,Porcentaje de conductores involucrados en colisiones fatales que no estaban distraídos,Porcentaje de conductores involucrados en colisiones fatales que no habían estado involucrados en accidentes previos,Primas de seguro de automóvil ($),Pérdidas incurridas por las compañías de seguros por colisiones por conductor asegurado ($)
0,Alabama,18.8,39,30.0,96,80.0,784.55,145.08
1,Alaska,18.1,41,25.0,90,94.0,1053.48,133.93
2,Arizona,18.6,35,28.0,84,96.0,899.47,110.35
3,Arkansas,22.4,18,26.0,94,95.0,827.34,142.39
4,California,12.0,35,28.0,91,89.0,878.41,165.63


## 2. Inspección inicial del dataset

En esta sección queremos responder rápidamente:

- ¿Cuántas filas y columnas tiene el dataset?
- ¿Qué tipo de dato tiene cada columna?
- ¿Hay valores faltantes?
- ¿Cómo lucen las estadísticas básicas de las columnas numéricas?



In [48]:
# Dimensiones del dataset
print("Dimensiones del dataset (filas, columnas):", df.shape)

# Información general de tipos de datos y valores no nulos
df.info()

Dimensiones del dataset (filas, columnas): (52, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                --------------  -----  
 0   Estado                                                                                                                52 non-null     object 
 1   Número de conductores involucrados en colisiones fatales por cada mil millones de millas                              52 non-null     float64
 2   Porcentaje de conductores involucrados en colisiones fatales con exceso de velocidad                                  52 non-null     int64  
 3   Porcentaje de conductores involucrados en colisiones fatales bajo la influencia del alcohol                

In [49]:
# Estadísticas descriptivas de las columnas numéricas
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Número de conductores involucrados en colisiones fatales por cada mil millones de millas,52.0,15.723077,4.109989,5.90,12.6500,15.450,18.4500,23.90
Porcentaje de conductores involucrados en colisiones fatales con exceso de velocidad,52.0,31.730769,9.538601,13.00,23.0000,34.000,38.0000,54.00
Porcentaje de conductores involucrados en colisiones fatales bajo la influencia del alcohol,51.0,30.941176,4.692171,23.00,28.0000,30.000,33.0000,44.00
Porcentaje de conductores involucrados en colisiones fatales que no estaban distraídos,52.0,85.961538,15.012363,10.00,83.5000,88.000,95.0000,100.00
Porcentaje de conductores involucrados en colisiones fatales que no habían estado involucrados en accidentes previos,51.0,88.647059,7.036543,76.00,82.5000,88.000,95.0000,100.00
Primas de seguro de automóvil ($),52.0,893.637500,182.993177,641.96,768.6900,860.075,1015.8225,1301.52
Pérdidas incurridas por las compañías de seguros por colisiones por conductor asegurado ($),52.0,134.791538,24.685192,82.75,114.7325,136.590,151.6750,194.78


### Preguntas rápidas (para discutir en voz alta / comentar en celdas Markdown):

1. ¿Te llama la atención algún rango de valores (mínimo, máximo, promedio) en las variables numéricas?
2. ¿Hay columnas con valores muy diferentes entre el mínimo y el máximo que podrían indicar presencia de **outliers**?
3. ¿Qué columnas parecen claramente numéricas y cuál es la única claramente categórica?


## 3. Renombrar columnas a un formato más manejable

Los nombres originales son largos. Vamos a renombrarlos a `snake_case` para facilitar el trabajo en Python.

Además, esto es una **buena práctica** para futuros modelos.


In [50]:
# Mostrar nombres de columnas originales
df.columns.tolist()

['Estado',
 'Número de conductores involucrados en colisiones fatales por cada mil millones de millas',
 'Porcentaje de conductores involucrados en colisiones fatales con exceso de velocidad',
 'Porcentaje de conductores involucrados en colisiones fatales bajo la influencia del alcohol',
 'Porcentaje de conductores involucrados en colisiones fatales que no estaban distraídos',
 'Porcentaje de conductores involucrados en colisiones fatales que no habían estado involucrados en accidentes previos',
 'Primas de seguro de automóvil ($)',
 'Pérdidas incurridas por las compañías de seguros por colisiones por conductor asegurado ($)']

In [51]:
# Diccionario de renombre de columnas

rename_dict = {
    "Estado": "estado",
    "Número de conductores involucrados en colisiones fatales por cada mil millones de millas": "conductores_fatales_mm_millas",
    "Porcentaje de conductores involucrados en colisiones fatales con exceso de velocidad": "pct_exceso_velocidad",
    "Porcentaje de conductores involucrados en colisiones fatales bajo la influencia del alcohol": "pct_alcohol",
    "Porcentaje de conductores involucrados en colisiones fatales que no estaban distraídos": "pct_no_distraidos",
    "Porcentaje de conductores involucrados en colisiones fatales que no habían estado involucrados en accidentes previos": "pct_sin_accidentes_previos",
    "Primas de seguro de automóvil ($)": "primas_seguro",
    "Pérdidas incurridas por las compañías de seguros por colisiones por conductor asegurado ($)": "perdidas_por_conductor",
}

# Aplicar el cambio
df = df.rename(columns=rename_dict)

# Verificar
df.head()

,estado,conductores_fatales_mm_millas,pct_exceso_velocidad,pct_alcohol,pct_no_distraidos,pct_sin_accidentes_previos,primas_seguro,perdidas_por_conductor
0,Alabama,18.8,39,30.0,96,80.0,784.55,145.08
1,Alaska,18.1,41,25.0,90,94.0,1053.48,133.93
2,Arizona,18.6,35,28.0,84,96.0,899.47,110.35
3,Arkansas,22.4,18,26.0,94,95.0,827.34,142.39
4,California,12.0,35,28.0,91,89.0,878.41,165.63


## 4. Tipos de datos, valores faltantes y duplicados

Antes de hacer modelos o gráficos, debemos asegurarnos de que:

- Cada columna tenga el **tipo de dato correcto** (numérico vs categórico).
- No haya **duplicados** problemáticos.
- Conozcamos la magnitud de los **valores faltantes**.


In [52]:
# 4.1 Revisar tipos de datos
df.dtypes

estado                            object
conductores_fatales_mm_millas    float64
pct_exceso_velocidad               int64
pct_alcohol                      float64
pct_no_distraidos                  int64
pct_sin_accidentes_previos       float64
primas_seguro                    float64
perdidas_por_conductor           float64
dtype: object

In [53]:
# Convertir `state` en tipo categórico (opcional pero recomendable)
df["estado"] = df["estado"].astype("category")

# Verificamos de nuevo
df.dtypes

estado                           category
conductores_fatales_mm_millas     float64
pct_exceso_velocidad                int64
pct_alcohol                       float64
pct_no_distraidos                   int64
pct_sin_accidentes_previos        float64
primas_seguro                     float64
perdidas_por_conductor            float64
dtype: object

In [54]:
# 4.2 Valores faltantes (conteo y porcentaje)
conteo_nulos = df.isna().sum()
nulos_pct = (conteo_nulos / len(df) * 100).round(2)

resumen_nulos = pd.DataFrame({
    "conteo_nulost": conteo_nulos,
    "nulos_pct": nulos_pct
}).sort_values(by="nulos_pct", ascending=False)

resumen_nulos

,conteo_nulost,nulos_pct
pct_alcohol,1,1.92
pct_sin_accidentes_previos,1,1.92
conductores_fatales_mm_millas,0,0.00
estado,0,0.00
pct_exceso_velocidad,0,0.00
pct_no_distraidos,0,0.00
primas_seguro,0,0.00
perdidas_por_conductor,0,0.00


In [55]:
# 4.3 Filas duplicadas
num_duplicates = df.duplicated().sum()
print(f"Número de filas duplicadas en el dataset: {num_duplicates}")

Número de filas duplicadas en el dataset: 1


### EJERCICIO 1 (para que programen ustedes)

1. Si existieran filas duplicadas, decidan si corresponde eliminarlas o no.  
2. Si existieran valores faltantes, definan una estrategia simple (por ejemplo, eliminar filas con NA en variables críticas).

> **Sugerencia**: usen `df.drop_duplicates()` y/o `df.dropna()` o imputación sencilla (media/mediana) si tiene sentido.


In [63]:
# EJERCICIO 1: completa la limpieza según las decisiones de tu grupo.
# A modo de ejemplo, aquí dejamos líneas comentadas:

# 1) Eliminar filas completamente duplicadas
df = df.drop_duplicates()

# 2) Si quisieran eliminar filas con NA en alguna columna específica:
# cols_criticas = ["drivers_fatal_per_billion_miles", "pct_speeding", "pct_alcohol"]
# df = df.dropna(subset=cols_criticas)

# Muestra la nueva forma del dataframe
df.shape

(51, 8)

## 5. Detección simple de outliers (z-score)

Ahora vamos a detectar **outliers** numéricos usando un criterio sencillo:

- Para cada columna numérica, calculamos el **z-score**.
- Marcamos como outlier aquellos registros con |z| > 3 en al menos una columna.

Esto no es perfecto, pero es un buen primer filtro.

In [64]:
from scipy import stats

# Seleccionar solo columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols

['conductores_fatales_mm_millas',
 'pct_exceso_velocidad',
 'pct_alcohol',
 'pct_no_distraidos',
 'pct_sin_accidentes_previos',
 'primas_seguro',
 'perdidas_por_conductor']

In [58]:
# Calcular z-scores para cada columna numérica (ignorando filas con NA)
z_scores = np.abs(stats.zscore(df[numeric_cols], nan_policy='omit'))

# z_scores es un array; construimos una máscara de outliers
outlier_mask = (z_scores > 3).any(axis=1)

print(f"Número de filas marcadas como outlier (|z|>3 en alguna variable numérica): {outlier_mask.sum()}")

# Mostrar algunas filas outlier para inspección
df[outlier_mask].head()

Número de filas marcadas como outlier (|z|>3 en alguna variable numérica): 2


,estado,conductores_fatales_mm_millas,pct_exceso_velocidad,pct_alcohol,pct_no_distraidos,pct_sin_accidentes_previos,primas_seguro,perdidas_por_conductor
24,Mississippi,17.6,15,31.0,10,100.0,896.07,155.77
50,Wisconsin,13.8,36,33.0,39,84.0,670.31,106.62


In [59]:
# Estadísticas descriptivas de las columnas numéricas
df.describe().T

,count,mean,std,min,25%,50%,75%,max
conductores_fatales_mm_millas,52.0,15.723077,4.109989,5.90,12.6500,15.450,18.4500,23.90
pct_exceso_velocidad,52.0,31.730769,9.538601,13.00,23.0000,34.000,38.0000,54.00
pct_alcohol,51.0,30.941176,4.692171,23.00,28.0000,30.000,33.0000,44.00
pct_no_distraidos,52.0,85.961538,15.012363,10.00,83.5000,88.000,95.0000,100.00
pct_sin_accidentes_previos,51.0,88.647059,7.036543,76.00,82.5000,88.000,95.0000,100.00
primas_seguro,52.0,893.637500,182.993177,641.96,768.6900,860.075,1015.8225,1301.52
perdidas_por_conductor,52.0,134.791538,24.685192,82.75,114.7325,136.590,151.6750,194.78


## 📌 Outliers en la variable `pct_no_distraidos`

Al analizar la distribución de esta variable, se identificaron dos valores que se alejan significativamente del rango típico:

- **Mississippi** → `pct_no_distraidos = 10`
- **Wisconsin** → `pct_no_distraidos = 39`

Estos valores son **outliers** porque están muy por debajo del resto de los estados y pueden afectar análisis estadísticos y modelos predictivos.


In [60]:
# EJERCICIO 2: decide qué hacer con los outliers.

# Ejemplo: eliminar outliers (si el grupo decide hacerlo)
# df_clean = df[~outlier_mask].copy()

# Ejemplo alternativo: mantenerlos, pero copiamos de todos modos
df_clean = df.copy()

df_clean.shape

(52, 8)

## 6. Creación de variables nuevas

Vamos a crear algunas variables derivadas que pueden ser útiles luego para modelos:

1. **ratio_losses_premiums**: qué proporción de las primas de seguro se “pierde” en siniestros.  
   \[
   \text{ratio\_losses\_premiums} = \frac{\text{losses\_per\_insured\_driver}}{\text{car\_insurance\_premiums}}
   \]

2. **risk_speed_alcohol**: indicador simple de riesgo combinando % de choques con exceso de velocidad y % con alcohol.  
   \[
   \text{risk\_speed\_alcohol} = \text{pct\_speeding} + \text{pct\_alcohol}
   \]

3. **pct_safe_behaviour**: porcentaje aproximado de conductores que en choques fatales *no estaban distraídos* y *no tenían accidentes previos*.  
   (Esto es una aproximación solo para fines didácticos.)


In [61]:
# 6.1 Crear ratio de pérdidas sobre primas
df_clean["ratio_losses_premiums"] = (
    df_clean["perdidas_por_conductor"] / df_clean["primas_seguro"]
)

# 6.2 Crear indicador combinado de riesgo (velocidad + alcohol)
df_clean["risk_speed_alcohol"] = (
    df_clean["pct_exceso_velocidad"] + df_clean["pct_alcohol"]
)

# 6.3 Aproximación de "comportamiento seguro" combinando no distraídos y sin accidentes previos
# Ojo: esto no es una fórmula oficial, solo una aproximación pedagógica
df_clean["pct_safe_behaviour"] = (
    df_clean["pct_no_distraidos"] * df_clean["pct_sin_accidentes_previos"] / 100.0
)

df_clean[[
    "estado",
    "primas_seguro",
    "perdidas_por_conductor",
    "ratio_losses_premiums",
    "risk_speed_alcohol",
    "pct_safe_behaviour"
]].head()


,estado,primas_seguro,perdidas_por_conductor,ratio_losses_premiums,risk_speed_alcohol,pct_safe_behaviour
0,Alabama,784.55,145.08,0.184921,69.0,76.80
1,Alaska,1053.48,133.93,0.127131,66.0,84.60
2,Arizona,899.47,110.35,0.122683,63.0,80.64
3,Arkansas,827.34,142.39,0.172106,44.0,89.30
4,California,878.41,165.63,0.188557,63.0,80.99


## 7. Selección de columnas para modelado

El objetivo del siguiente notebook será construir un **modelo de regresión lineal**.  
Una opción razonable es predecir:

- **Target (Y):** `losses_per_insured_driver`  
- **Features (X):** variables que describen el riesgo de choques y las primas.

Ejemplo de columnas para el modelo:

- `drivers_fatal_per_billion_miles`
- `pct_speeding`
- `pct_alcohol`
- `pct_not_distracted`
- `pct_no_previous_accidents`
- `car_insurance_premiums`
- `ratio_losses_premiums`
- `risk_speed_alcohol`
- (y la variable nueva que ustedes definan)


In [62]:
# ==========================
# 3. Seleccionar X e y 
# ==========================

# Variable objetivo (lo que queremos predecir)
target_col = "primas_seguro"

# Variables predictoras (características del modelo)
feature_cols = [
    "conductores_fatales_mm_millas",
    "pct_exceso_velocidad",
    "pct_alcohol",
    "pct_no_distraidos",
    "pct_sin_accidentes_previos",
    "perdidas_por_conductor"
]

# Creamos un DataFrame con solo las columnas necesarias
df_model = df_clean[feature_cols + [target_col]].copy()

df_model.head()



,conductores_fatales_mm_millas,pct_exceso_velocidad,pct_alcohol,pct_no_distraidos,pct_sin_accidentes_previos,perdidas_por_conductor,primas_seguro
0,18.8,39,30.0,96,80.0,145.08,784.55
1,18.1,41,25.0,90,94.0,133.93,1053.48
2,18.6,35,28.0,84,96.0,110.35,899.47
3,22.4,18,26.0,94,95.0,142.39,827.34
4,12.0,35,28.0,91,89.0,165.63,878.41


## 8. Checkpoint final del Notebook 1

Al terminar este notebook deberías tener:

- Un dataset con nombres de columnas limpios (`snake_case`).
- Tipos de datos adecuados (estado como categoría, resto numérico).
- Revisión (y decisión) sobre duplicados, valores faltantes y outliers.
- Nuevas variables derivadas con sentido de negocio.
- Un dataframe `model_df` listo para usar en el **Notebook 2 – Regresión Lineal**.

### Preguntas de reflexión (para 5 minutos finales)

1. ¿Qué variable crees que tendrá mayor impacto en las pérdidas de las aseguradoras (`losses_per_insured_driver`) y por qué?  
2. ¿Qué decisiones de limpieza podrían cambiar fuertemente los resultados de un futuro modelo?  
3. Si tuvieras que explicarle a alguien de negocio lo que hiciste en este notebook, ¿cómo lo resumirías en 3 frases?

En la próxima sesión vamos a:

- Dividir `model_df` en conjuntos de entrenamiento y prueba.
- Entrenar un modelo de **regresión lineal**.
- Evaluar métricas (MAE, RMSE, R²).
- Analizar la importancia de las variables.

Guarda este notebook: lo vas a reutilizar directamente.
